In [8]:
from Cryptodome.Random import random
from Cryptodome.Cipher import AES
from Cryptodome.PublicKey import RSA
from hashlib import sha256
import os
import base64

def pad(msg):
    n = 16 - len(msg) % 16
    return msg + bytes([n]) * n

def unpad(msg):
    assert len(msg) > 0 and len(msg) % 16 == 0
    n = msg[-1]
    assert 1 <= n <= 16
    assert msg[-n:] == bytes([n]) * n
    return msg[:-n]

ca_key=(123911229913422142875264410911959753429255609781060012540592865384481400430784339515602026823919372067245349130530370079412035001372557207523878469898937327849288805682100737656179353201361997603827751534685210816357362467822736133251729781421390068496293277151621728336136046748401714936280146300069794792289, 65537)

def cert_verify(mess):
    try:
        gm,cs=mess.split('_')
        rl=bytes.fromhex(gm)
        ps=int(cs,16)
        isign=int.from_bytes(sha256(rl).digest(),'big')
        if pow(ps,ca_key[1],ca_key[0])==isign:
            return (True,rl)
        return (False,b"")
    except:
        print("ERROR!")
        return (False,b"")

In [9]:
#之前获取的两个cert，不过实际上没用。
acert='416c696365_7d2e831fc7c37b4932604c66de52a3c7f631eee0badffb42f40899fb88ff19e41518fc8326f620aaeb54cecd8bd22a5fa8576848c131370c0fcb8fb0e093fbd0d76b73c47c04ffe2b913472595c9987d53b8cdffa4ff16f6a6176f215e6ded63e65089d763691a4cef91284e6934b95580e64854304b99e87215b582e201222d'
bcert='426f62_2453e551e17519b6dcfdae46f75b9d92dd943fcb5d2c71911adb83f43753dfba86be013dfb8c2413cbc70eac852e312040d3fbf471b57da2ae7a7aa95d39f3b972e34214f5cc69093d9929b2c5fc24684b0d5443dcb8338d1f52bb34f9913bd1e98c29302bbbd3cb532001adde7f06d93badcdee7b12b1b550a816537e950bad'
P=90217665064898209874945440415607645619739552209209829698859302616784804764535733761793578496057231124307702060703777047113648993244620436073319613409562635176198907683514151487667596066485528729421061091099275256268502121631878101098253283229895343146346757918637948217208979264467987857303672675468072062419
G=13


In [10]:
#d=0
d=2
hex(pow(G,d,P))

'0xa9'

In [11]:
key=pow(int("11db0876a1a4073984154312a47354de1a05d3984a674e776e5b1b5b068d261c2afeb1a4f9c71c23782bb8b413450530263c163d31c6087e71e4531513d5510afcb435239bc6395dfba1edd1f218236236f0c0b21d72260e61332f95ac77812202d82927e4568cf612a8691ed054688b89c08384e99a63bb9de5ec4a38302ffc",16),d,P)
aes_key=int.to_bytes(key%(2**128),16,'big')

In [12]:
key<(2**512)

False

In [13]:
##生成第二轮通信内容；但其实没用。
aes_iv = os.urandom(16)
aes=AES.new(aes_key, AES.MODE_CBC, aes_iv)
cmess=(aes_iv+aes.encrypt(pad(bcert.encode()))).hex()
cmess

'c62730f3eb09827590ec7ecace027bf6c36467588f1ce874a44ba3a704ffeb238039a429f3519be5df34061ac4ae893aaf797503d195e0bf7d09c3d465f1198910a03e86d13cf9daa978ffd05d473fa1ba3421f00b1469f7ac08249e1a13e0e589cedc7f71ffa90f78d5ed5b223ab8bb89b62744358d489ab2feb40f6e70895fb19db075ed64b2781f1bde105587d8b715db84a05c37e5f88b43ddc421ad2758553109a3bc23fb39f58af0507bd5cb10b71bdb04d904fd8b123f9742bab9b57b5abd87caac13657ad1ae90c03f587f2f0a18a55079f19b4d14a156b9bb03558d552267e223de518c3de01e9658a51f42392fea7ce52bca30ceebdbd165d900fdebcdca999770c94b412f657c50df92a98078fc82daa5f25ff8fe86b770361bf9'

In [15]:
cflag=bytes.fromhex("12de22744a52d06a61d448c3cfa06f9c6c1dee4166f177982fb5811a9fb30c0411c6b072ed1f54b1551f3fd9f2ed290df2f4ff8df1ec5a487c56f2af59fb3fb9")
civ,flagc=cflag[:16],cflag[16:]
caes=AES.new(aes_key,AES.MODE_CBC,civ)
flag=unpad(caes.decrypt(flagc)).decode()
flag

'flag{Wh4t_A_w3Ak_Pr0toCo1_9ffbb9f4}'